<a href="https://colab.research.google.com/github/Bhaktaraj-Pooja/ML-Project/blob/main/TAN_Traffic_prediction2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [160]:
import pandas as pd

from sklearn import tree
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder

In [161]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#**Load data**

In [162]:
train_set = pd.read_csv('/content/drive/MyDrive/Train_traffic.csv')
test_set = pd.read_csv('/content/drive/MyDrive/Test_traffic.csv')

In [163]:
print(train_set.head())

             date_time is_holiday  ...  weather_description  traffic_volume
0  2012-10-02 09:00:00       None  ...     scattered clouds            5545
1  2012-10-02 10:00:00       None  ...        broken clouds            4516
2  2012-10-02 11:00:00       None  ...      overcast clouds            4767
3  2012-10-02 12:00:00       None  ...      overcast clouds            5026
4  2012-10-02 13:00:00       None  ...        broken clouds            4918

[5 rows x 15 columns]


#**Prepare Train Data**

In [164]:
train_set['date_time'] = pd.to_datetime(train_set.date_time)

In [165]:
train_set['year'] = train_set.date_time.dt.year
train_set['month'] = train_set.date_time.dt.month
train_set['day'] = train_set.date_time.dt.day
train_set['hour'] = train_set.date_time.dt.hour

In [166]:
train_copy = train_set.drop(['date_time'], axis=1)
print(train_copy.head())

  is_holiday  air_pollution_index  humidity  wind_speed  ...  year  month  day  hour
0       None                  121        89           2  ...  2012     10    2     9
1       None                  178        67           3  ...  2012     10    2    10
2       None                  113        66           3  ...  2012     10    2    11
3       None                   20        66           3  ...  2012     10    2    12
4       None                  281        65           3  ...  2012     10    2    13

[5 rows x 18 columns]


#**One-hot-encoding**

In [167]:
train_onehot = train_copy.copy()

In [168]:
train_onehot = pd.get_dummies(train_onehot, columns=['is_holiday', 'weather_type', 'weather_description'], 
                              prefix=['is_holiday', 'weather_type', 'weather_desc'])

In [169]:
train_onehot = train_onehot.astype(float)
train_onehot.head()

,air_pollution_index,humidity,wind_speed,wind_direction,visibility_in_miles,dew_point,temperature,rain_p_h,snow_p_h,clouds_all,traffic_volume,year,month,day,hour,is_holiday_Christmas Day,is_holiday_Columbus Day,is_holiday_Independence Day,is_holiday_Labor Day,is_holiday_Martin Luther King Jr Day,is_holiday_Memorial Day,is_holiday_New Years Day,is_holiday_None,is_holiday_State Fair,is_holiday_Thanksgiving Day,is_holiday_Veterans Day,is_holiday_Washingtons Birthday,weather_type_Clear,weather_type_Clouds,weather_type_Drizzle,weather_type_Fog,weather_type_Haze,weather_type_Mist,weather_type_Rain,weather_type_Smoke,weather_type_Snow,weather_type_Squall,weather_type_Thunderstorm,weather_desc_SQUALLS,weather_desc_Sky is Clear,weather_desc_broken clouds,weather_desc_drizzle,weather_desc_few clouds,weather_desc_fog,weather_desc_freezing rain,weather_desc_haze,weather_desc_heavy intensity drizzle,weather_desc_heavy intensity rain,weather_desc_heavy snow,weather_desc_light intensity drizzle,weather_desc_light intensity shower rain,weather_desc_light rain,weather_desc_light rain and snow,weather_desc_light shower snow,weather_desc_light snow,weather_desc_mist,weather_desc_moderate rain,weather_desc_overcast clouds,weather_desc_proximity shower rain,weather_desc_proximity thunderstorm,weather_desc_proximity thunderstorm with drizzle,weather_desc_proximity thunderstorm with rain,weather_desc_scattered clouds,weather_desc_shower drizzle,weather_desc_shower snow,weather_desc_sky is clear,weather_desc_sleet,weather_desc_smoke,weather_desc_snow,weather_desc_thunderstorm,weather_desc_thunderstorm with drizzle,weather_desc_thunderstorm with heavy rain,weather_desc_thunderstorm with light drizzle,weather_desc_thunderstorm with light rain,weather_desc_thunderstorm with rain,weather_desc_very heavy rain
0,121.0,89.0,2.0,329.0,1.0,1.0,288.28,0.0,0.0,40.0,5545.0,2012.0,10.0,2.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,178.0,67.0,3.0,330.0,1.0,1.0,289.36,0.0,0.0,75.0,4516.0,2012.0,10.0,2.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,113.0,66.0,3.0,329.0,2.0,2.0,289.58,0.0,0.0,90.0,4767.0,2012.0,10.0,2.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,20.0,66.0,3.0,329.0,5.0,5.0,290.13,0.0,0.0,90.0,5026.0,2012.0,10.0,2.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,281.0,65.0,3.0,329.0,7.0,7.0,291.14,0.0,0.0,75.0,4918.0,2012.0,10.0,2.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#**Preparing the Test data**

In [170]:
test_set['date_time'] = pd.to_datetime(test_set.date_time)

In [171]:
test_set['year'] = test_set.date_time.dt.year
test_set['month'] = test_set.date_time.dt.month
test_set['day'] = test_set.date_time.dt.day
test_set['hour'] = test_set.date_time.dt.hour

In [172]:
test_copy = test_set.drop(['date_time'], axis=1)

In [173]:
test_onehot = test_copy.copy()
test_onehot = pd.get_dummies(test_onehot, columns=['is_holiday', 'weather_type', 'weather_description'], 
                              prefix=['is_holiday', 'weather_type', 'weather_desc'])

In [174]:
print(len(train_onehot.columns))
print(len(train_onehot))

76
33750


In [175]:
print(len(test_onehot.columns))
print(len(test_onehot))

68
14454


In [176]:
# find the features that are not in test data set
for x in train_onehot.columns:
    if x not in test_onehot.columns and x != 'traffic_volume':
        print(x)
        test_onehot[x] = train_onehot[x]

weather_type_Squall
weather_desc_SQUALLS
weather_desc_freezing rain
weather_desc_light rain and snow
weather_desc_shower snow
weather_desc_thunderstorm with drizzle
weather_desc_very heavy rain


In [177]:
test_onehot = test_onehot.astype(float)

#**Train and Test data**

In [178]:
y_train = train_onehot['traffic_volume']
x_train = train_onehot.drop(['traffic_volume'], axis=1)

#**Training**

In [179]:
dec_tree_reg = tree.DecisionTreeRegressor()
dec_tree_reg.fit(x_train, y_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

#**Decision Tree Regressor**

In [180]:
preds = dec_tree_reg.predict(test_onehot)
print(len(preds))
print(preds)
preds = preds.astype(int)
print(preds)

14454
[ 582.  582.  582. ... 3382. 1422. 1011.]
[ 582  582  582 ... 3382 1422 1011]


In [181]:
submission = pd.DataFrame(columns = ['date_time', 'traffic_volume']) 
print(submission.head())
submission.date_time = test_set.date_time
submission.traffic_volume = preds
print(len(submission))
print(submission.head())
submission.to_csv('/content/drive/MyDrive/traffic_predict.csv', index=False)

Empty DataFrame
Columns: [date_time, traffic_volume]
Index: []
14454
            date_time  traffic_volume
0 2017-05-18 00:00:00             582
1 2017-05-18 00:00:00             582
2 2017-05-18 00:00:00             582
3 2017-05-18 01:00:00             355
4 2017-05-18 01:00:00             870
